In [4]:
import asyncio
from datetime import datetime
from pathlib import Path
import subprocess
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from google.cloud import storage

from playwright.async_api import (
    async_playwright,
    expect,
    TimeoutError as PlaywrightTimeoutError,
)

In [5]:
data = {
    "userid": [
    "10002536", "10008847", "10004679", "10023836",
    "10033752", "10038584", "10044023", "10192623",
    "10165926", "10093976", "10132431"
    ],
    "名前": [
        "キンシュウサイ", "大岡正樹", "池海龍（イケカイリュウ）", "飯島美桜",
        "江原ケイト", "樋口大輔", "加藤順平", "加藤奈津子",
        "松本明子", "石崎卓", "玉野利家"
    ],
    "PW": [
        "Kin20240301", "Hasegawa110", "Chihailong0803", "Cherry2525",
        "Tennis2784!", "46495963¥@Desu", "WCbifd@3NANwXGE", "aDA8a6MWs3XEz_Y",
        "Saikou1234", "Zaitaku2023", "Packen0731t"
    ]
}

user_df = pd.DataFrame(data)

In [ ]:
URL = "https://kouen.sports.metro.tokyo.lg.jp/web/index.jsp"
BLOCK_TEXT = "しばらく経ってから"

async def run(user_df):
    p = await async_playwright().start()

    context = await p.chromium.launch_persistent_context(
        user_data_dir="./debug-profile",
        channel="chrome",
        headless=False,
    )

    page = await context.new_page()

    all_results = []

    for _, row in user_df.iterrows():
        userid = str(row["userid"])
        pw = str(row["PW"])

        print(f"Processing user: {userid}")

        await page.goto(URL)

        # --- wait until not blocked ---
        while True:
            content = await page.content()
            if BLOCK_TEXT not in content:
                break
            print("Blocked. Waiting 5 seconds...")
            await asyncio.sleep(5)
            await page.reload()

        # --- login ---
        await page.click("text=ログイン")
        await page.fill("#userId", userid)
        await page.fill("#password", pw)

        async with page.expect_navigation(wait_until="networkidle"):
            await page.click("#btn-go")

        # If login failed, login form will still exist
        if await page.locator("#userId").count():
            all_results.append([userid, "login_fail", "login_fail", "login_fail", "login_fail", "login_fail", "login_fail"])
            continue

        # --- go to lottery result ---
        await page.click('a[data-target="#modal-menus"]')
        await page.get_by_role("link", name="抽選結果", exact=True).click()
        selector = "article#lottery-result table:has(caption:has-text('確認されていない当選結果'))"

        try:
            await page.wait_for_selector(selector, timeout=5000)
            table1 = page.locator(selector).first

            rows = await table1.locator("tbody tr").all()

            data_won = []
            for r in rows:
                tds = await r.locator("td").all()
                row_vals = [(await td.inner_text()).strip() for td in tds]
                data_won.append(row_vals)
                df_won = pd.DataFrame(
                    data_won,
                    columns=["公園・施設", "利用日", "利用時間", "利用料金", "col5"][: len(data_won[0])] if data_won else None
                )
                df_won.drop(columns=["col5"], inplace=True, errors='ignore')
                df_won["user_id"] = userid
                df_won["table_type"] = "won"
                df_won["状態"] = "当選"
                df_won["予約番号"] = None
                all_results.append(df_won)

        except PlaywrightTimeoutError:
            print("No won table found")
      
        await page.click("#lotconfbtn-go")
        await page.wait_for_selector("article#lottery-result table")
        rows = await page.locator("article#lottery-result table tr").all()
        data_lost = []
        for r in rows:
            cells = await r.locator("th, td").all_inner_texts()
            data_lost.append([c.strip() for c in cells])

        if len(data_lost) > 1:
            header = data_lost[0]
            body = data_lost[1:]
            df_lost = pd.DataFrame(body, columns=header)
            df_lost["user_id"] = userid
            df_lost["table_type"] = "lost"
            all_results.append(df_lost)

        # --- logout (important) ---
        await page.goto(URL)

    await context.close()
    await p.stop()
    return pd.concat(all_results, ignore_index=True)

# Run
final_df = await run(user_df)


Processing user: 10002536
No won table found
Processing user: 10008847


NameError: name 'result_lst' is not defined

In [ ]:
# # Generate date string
# today_str = datetime.today().strftime("%Y%m%d")

# # Build path safely
# base_path = Path.home() / "workspace" / "熊猫カンパニー" / "lottery_results"
# base_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
# file_path = base_path / f"lottery_results_{today_str}.xlsx"

# # Export
# final_df.to_excel(file_path, index=False)

In [ ]:
local_folder = Path.home() / "workspace" / "熊猫カンパニー" / "reservation"
local_folder.mkdir(parents=True, exist_ok=True)

remote_folder = "googledrive:熊猫カンパニー/reservation"

today_str = datetime.today().strftime("%Y%m%d")
file_path = local_folder / f"lottery_results_{today_str}.xlsx"

final_df.to_excel(file_path, index=False)

subprocess.run(
    ["paplay", "/usr/share/sounds/freedesktop/stereo/complete.oga"],
    check=False
)

subprocess.run(
    ["rclone", "copy", str(file_path), remote_folder, "-P"],
    check=True
)